In [1]:
from neo4j import GraphDatabase
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm


In [3]:

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "12345678"))

def get_related_keywords(keyword):
    query = """
    MATCH (n)
    WHERE toLower(n.name) CONTAINS toLower($keyword)
    MATCH (n)--(related)
    RETURN DISTINCT related.name AS keyword, labels(related) AS type
    """
    with driver.session() as session:
        result = session.run(query, keyword=keyword)
        return [record["keyword"] for record in result]



In [4]:
print(get_related_keywords("chauffage"))

['Pompe à chaleur', 'Plancher chauffant', 'Isolation thermique']


In [2]:
df = pd.read_csv("CHUNKS\\file_chunks.csv") 
model = SentenceTransformer("dangvantuan/sentence-camembert-base")

# Let's quickly check the embedding dimension:
dummy_vector = model.encode("texte de test")
dim = len(dummy_vector)  # This will be used in Qdrant's vector params
print(f"Embedding dimension: {dim}")

Embedding dimension: 768


In [5]:
# Adjust host/port if Qdrant is elsewhere:
client = QdrantClient(url="http://localhost:6333")  # or QdrantClient("localhost", port=6333)

# Create or recreate a collection
collection_name = "PROJET_RCRA2"
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=dim, distance=Distance.COSINE)
)

# Now, embed and upsert each CSV chunk as a separate point in Qdrant
points_to_upsert = []
for idx, row in tqdm(df.iterrows()):
    text_chunk = row["chunk"]
    file_id = row["file"]  # might be useful metadata
    vector = model.encode(text_chunk)

    # Create a unique ID for each chunk row
    point_id = idx  
    payload = {
        "file": file_id, 
        "chunk": text_chunk
    }

    # Construct a PointStruct
    points_to_upsert.append(
        PointStruct(
            id=point_id,
            vector=vector.tolist(),
            payload=payload
        )
    )

C:\Users\amine\AppData\Local\Temp\ipykernel_17252\3964713409.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(
11630it [03:17, 58.87it/s]


In [6]:
BATCH_SIZE = 100  # you can tweak this

for i in range(0, len(points_to_upsert), BATCH_SIZE):
    batch = points_to_upsert[i:i + BATCH_SIZE]
    client.upsert(
        collection_name=collection_name,
        points=batch
    )
    print(f"✅ Uploaded batch {i} to {i + len(batch)} from {len(points_to_upsert)}")


✅ Uploaded batch 0 to 100 from 11630
✅ Uploaded batch 100 to 200 from 11630
✅ Uploaded batch 200 to 300 from 11630
✅ Uploaded batch 300 to 400 from 11630
✅ Uploaded batch 400 to 500 from 11630
✅ Uploaded batch 500 to 600 from 11630
✅ Uploaded batch 600 to 700 from 11630
✅ Uploaded batch 700 to 800 from 11630
✅ Uploaded batch 800 to 900 from 11630
✅ Uploaded batch 900 to 1000 from 11630
✅ Uploaded batch 1000 to 1100 from 11630
✅ Uploaded batch 1100 to 1200 from 11630
✅ Uploaded batch 1200 to 1300 from 11630
✅ Uploaded batch 1300 to 1400 from 11630
✅ Uploaded batch 1400 to 1500 from 11630
✅ Uploaded batch 1500 to 1600 from 11630
✅ Uploaded batch 1600 to 1700 from 11630
✅ Uploaded batch 1700 to 1800 from 11630
✅ Uploaded batch 1800 to 1900 from 11630
✅ Uploaded batch 1900 to 2000 from 11630
✅ Uploaded batch 2000 to 2100 from 11630
✅ Uploaded batch 2100 to 2200 from 11630
✅ Uploaded batch 2200 to 2300 from 11630
✅ Uploaded batch 2300 to 2400 from 11630
✅ Uploaded batch 2400 to 2500 from 11

In [12]:
def expand_query_with_neo4j(user_query):
    """
    Naive approach: split user query into tokens 
    and gather related keywords from Neo4j.
    """
    tokens = user_query.lower().split()
    all_expanded = set()
    for token in tokens:
        expansions = get_related_keywords(token)
        all_expanded.update(expansions)
    return list(all_expanded)


In [13]:
def retrieve_relevant_chunks(user_query, expansions, top_k=5):
    """
    - Combine the user query with expansions
    - Embed once
    - Query Qdrant for the top_k most similar chunks
    """
    if expansions:
        expansion_text = " ".join(expansions)
        augmented_text = user_query + " " + expansion_text
    else:
        augmented_text = user_query
    
    query_vector = model.encode(augmented_text)

    # Qdrant query
    results = client.query_points(
        collection_name=collection_name,
        query=query_vector.tolist(),
        limit=top_k
    )
    
    # 'results' will be a list of ScoredPoint objects
    # Each contains 'payload' (with our stored chunk)
    return results


In [ ]:
def expand_query(user_query, top_k=5):
    # 1) Expand user query with Neo4j
    expansions = expand_query_with_neo4j(user_query)
    # 2) Retrieve relevant chunks from Qdrant
    docs = retrieve_relevant_chunks(user_query, expansions, top_k=top_k)
    context_text = ""
    for doc in docs:
        payload = doc[1][0].payload
        context_text += f"Fichier: {payload['file']}\nChunk: {payload['chunk']}\n\n"
    full_prompt = f"""
    CONTEXTE:
    {context_text}

    QUESTION UTILISATEUR:
    {user_query}

    RÉPONSE ATTENDUE (en utilisant uniquement les informations ci-dessus) :
    """
    return full_prompt


In [32]:
query = "Quels sont les avantages d’utiliser une pompe à chaleur avec un plancher chauffant ?"
query = expand_query(query, top_k=3)

['Ventilation', 'Chauffage', 'Isolation thermique', "Membrane d'étanchéité", 'Chauffe-eau thermodynamique', 'RE2020', 'VMC double flux', 'Pompe à chaleur', 'Plancher chauffant', 'Énergie renouvelable']


In [28]:
import requests

def query_ollama(prompt, model="llama3.2"):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": model,
            "prompt": prompt,
            "stream": False  # you can use True if you want streamed responses
        }
    )
    data = response.json()
    return data["response"]




In [30]:

result = query_ollama(query)
print(result)

Les avantages de l'utilisation d'une pompe à chaleur avec un plancher chauffant sont :

1. Chauffage thermodynamique basse température : Cette technologie permet un chauffage efficace et respectueux de l'environnement, en utilisant les énergies renouvelables pour chauffer le plancher.
2. Suppression du bouclage ECS (Équilibreur de Charge Solaire) : En utilisant une pompe à chaleur avec un plancher chauffant, il n'est pas nécessaire d'installer un système de bouclage ECS, ce qui réduit les coûts et l'importance des déchets thermiques.
3. Économie d'énergie : Le chauffage basse température peut réduire la consommation d'énergie pour chauffer le plancher, ce qui peut entraîner une réduction des factures de gaz ou d'électricité.
4. Sécurité et confort : L'utilisation d'une pompe à chaleur avec un plancher chauffant peut améliorer la sécurité et le confort dans les bâtiments, en particulier pour les personnes âgées ou handicapées.
5. Double ou triple service : Cette technologie permet de fo